There are two methods for collecting data: one using MediaPipe landmarks and the other using traditional techniques. Please find the respective code for both methods below.

1. To collect data using the traditional method, please use the code in the second cell.
2. To collect data using MediaPipe landmarks, please use the code in the third cell.

In [1]:
# import cv2
# import os

# # Main directory for storing gesture images
# main_directory = r'C:\Sign_Language_Detection\gesture_data_set'

# # Mapping numerical keys to gestures
# gesture_keys = {
#     '1': 'call',
#     '2': 'dislike',
#     '3': 'fist',
#     '4': 'four',
#     '5': 'like',
#     '6': 'mute',
#     '7': 'ok',
#     '8': 'palm',
#     '9': 'peace',
#     '0': 'rock',
#     '.': 'stop'
# }

# # Create main directory if it doesn't exist
# if not os.path.exists(main_directory):
#     os.mkdir(main_directory)

# # Create subdirectories for each gesture
# for gesture in gesture_keys.values():
#     gesture_path = os.path.join(main_directory, gesture)
#     if not os.path.exists(gesture_path):
#         os.mkdir(gesture_path)

# # Start capturing video from webcam
# cap = cv2.VideoCapture(0)

# while True:
#     _, frame = cap.read()

#     # Count the number of images in each gesture directory
#     count = {gesture: len(os.listdir(os.path.join(main_directory, gesture))) for gesture in gesture_keys.values()}

#     row = frame.shape[1]
#     col = frame.shape[0]
#     cv2.rectangle(frame, (0, 40), (300, 300), (255, 255, 255), 2)
#     cv2.imshow("data", frame)
#     frame = frame[40:300, 0:300]
#     cv2.imshow("ROI", frame)
#     frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     frame = cv2.resize(frame, (48, 48))

#     interrupt = cv2.waitKey(10)

#     # Save images based on key press
#     if interrupt != -1:
#         key = chr(interrupt & 0xFF)
#         if key in gesture_keys:
#             gesture = gesture_keys[key]
#             cv2.imwrite(os.path.join(main_directory, gesture, f'{count[gesture]}.jpg'), frame)

#     # Break the loop on pressing 'ESC' key
#     if interrupt & 0xFF == 27:
#         break

# cap.release()
# cv2.destroyAllWindows()


In [9]:
import cv2
import mediapipe as mp
import os
import numpy as np

# Setup MediaPipe
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

# Main directory for storing gesture data
main_directory = r'C:\Sign_Language_Detection\gesture_data_set'

# Mapping numerical keys to gestures
gesture_keys = {
    '1': 'call',
    '2': 'dislike',
    '3': 'fist',
    '4': 'four',
    '5': 'like',
    '6': 'mute',
    '7': 'ok',
    '8': 'palm',
    '9': 'peace',
    '0': 'rock',
    '.': 'stop'
}

# Create main directory if it doesn't exist
if not os.path.exists(main_directory):
    os.mkdir(main_directory)

# Create subdirectories for each gesture
for gesture in gesture_keys.values():
    gesture_path = os.path.join(main_directory, gesture)
    if not os.path.exists(gesture_path):
        os.mkdir(gesture_path)

# Start capturing video from webcam
cap = cv2.VideoCapture(0)

with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            continue

        # Flip the image horizontally for a later selfie-view display
        image = cv2.flip(image, 1)
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image)

        # Draw the hand annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    image,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style())
        
        # Flip the image horizontally for a selfie-view display
        cv2.imshow('MediaPipe Hands', image)

        interrupt = cv2.waitKey(10)
        
        # Save landmarks data based on key press
        if interrupt != -1:
            key = chr(interrupt & 0xFF)
            if key in gesture_keys and results.multi_hand_landmarks:
                gesture = gesture_keys[key]
                gesture_path = os.path.join(main_directory, gesture)
                count = len(os.listdir(gesture_path))
                
                # Save landmarks data
                for hand_landmarks in results.multi_hand_landmarks:
                    landmarks = []
                    for landmark in hand_landmarks.landmark:
                        landmarks.append([landmark.x, landmark.y, landmark.z])
                    landmarks = np.array(landmarks)
                    np.save(os.path.join(gesture_path, f'{count}.npy'), landmarks)

        # Break the loop on pressing 'ESC' key
        if interrupt & 0xFF == 27:
            break

cap.release()
cv2.destroyAllWindows()

In [1]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("CUDA device count:", torch.cuda.device_count())
if torch.cuda.is_available():
    print("CUDA current device:", torch.cuda.current_device())
    print("CUDA device name:", torch.cuda.get_device_name(torch.cuda.current_device()))


CUDA available: False
CUDA device count: 0


AssertionError: Torch not compiled with CUDA enabled